In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from statsmodels.stats.weightstats import ztest
df = pd.read_csv("/content/Dry_Eye_Dataset.csv")
label_encoders = {}
for col in df.columns:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
X = df.drop(columns=['Dry Eye Disease']) # Replace 'class' with the actual target column name
y = df['Dry Eye Disease'] # Replace 'class'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": SVC(probability=True),
    "KNN": KNeighborsClassifier()
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type_i_error = fp / (fp + tn)
    type_ii_error = fn / (fn + tp)

    results[name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "Type I Error": type_i_error,
        "Type II Error": type_ii_error
    }
results_df = pd.DataFrame(results).T
print(results_df)
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
misclassified_indices = (y_test != y_pred)
correctly_classified_ages = X_test.loc[~misclassified_indices, "Age"]
misclassified_ages = X_test.loc[misclassified_indices, "Age"]
z_stat, p_value = ztest(correctly_classified_ages, misclassified_ages)
print(f"Z-Test for Mean Age Difference: Z-Statistic={z_stat}, P-Value={p_value}")
rf_fp_rate = results["Random Forest"]["Type I Error"]
if rf_fp_rate > 0.20:
    z_stat, p_value = ztest([rf_fp_rate], value=0.20)
    print(f"Z-Test for Type I Error in Random Forest: Z-Statistic={z_stat}, P-Value={p_value}")
z_stat, p_value = ztest([results["SVM"]["Type II Error"]], [results["KNN"]["Type II Error"]])
print(f"Z-Test for Type II Error between SVM and KNN: Z-Statistic={z_stat}, P-Value={p_value}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                     Accuracy  Precision    Recall  Type I Error  \
Logistic Regression   0.67325   0.673250  1.000000      1.000000   
Decision Tree         0.56425   0.687599  0.646491      0.605203   
Random Forest         0.69950   0.712091  0.929447      0.774292   
Gradient Boosting     0.70100   0.712461  0.932046      0.775057   
SVM                   0.67325   0.673250  1.000000      1.000000   
KNN                   0.59000   0.671220  0.766431      0.773527   

                     Type II Error  
Logistic Regression       0.000000  
Decision Tree             0.353509  
Random Forest             0.070553  
Gradient Boosting         0.067954  
SVM                       0.000000  
KNN                       0.233569  
Z-Test for Mean Age Difference: Z-Statistic=0.6699948763891681, P-Value=0.5028610563663367
Z-Test for Type I Error in Random Forest: Z-Statistic=nan, P-Value=nan
Z-Test for Type II Error between SVM and KNN: Z-Statistic=nan, P-Value=nan


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/statsmodels/stats/weightstats.py:1559: RuntimeWarning: invalid value encountered in scalar divide
  var = x1_var / (nobs1 - ddof)
/usr/local/lib/python3.11/dist-packages/statsmodels/stats/weightstats.py:1554: RuntimeWarning: invalid value encountered in scalar divide
  var /= nobs1 + nobs2 - 2 * ddof
